<a href="https://colab.research.google.com/github/sandeepgundeboina/SmallApps/blob/main/BroadCastPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [ ]:
transactions=[
    ('Apple',100,250,1),
    ('Samsung',200,None,1),
    ('mi',200,150,3),
    ('OnePlus',300,140,2),
    ('Oppo',200,170,2),
    ('Realme',300,None,4),
    ('Iqoo',400,170,4)
]
transDF=spark.createDataFrame(transactions,['product','quantity','price','store'])
transDF.show()

+-------+--------+-----+-----+
|product|quantity|price|store|
+-------+--------+-----+-----+
|  Apple|     100|  250|    1|
|Samsung|     200| NULL|    1|
|     mi|     200|  150|    3|
|OnePlus|     300|  140|    2|
|   Oppo|     200|  170|    2|
| Realme|     300| NULL|    4|
|   Iqoo|     400|  170|    4|
+-------+--------+-----+-----+



In [ ]:
transDF=transDF.repartition(5)
transDF.rdd.getNumPartitions()

5

In [ ]:
store=[
    (1,'Allahabad'),
    (2,'Bhemghal'),
    (3,'Chagal'),
    (4,'Delhi')
]
storeDF=spark.createDataFrame(store,['store_id','store_name'])
storeDF.show()

+--------+----------+
|store_id|store_name|
+--------+----------+
|       1| Allahabad|
|       2|  Bhemghal|
|       3|    Chagal|
|       4|     Delhi|
+--------+----------+



In [ ]:
from pyspark.sql.functions import broadcast
location_df=transDF.join(broadcast(storeDF),transDF.store==storeDF.store_id).select('product','price','quantity','store_name').sort('quantity')
location_df.show()

+-------+-----+--------+----------+
|product|price|quantity|store_name|
+-------+-----+--------+----------+
|  Apple|  250|     100| Allahabad|
|     mi|  150|     200|    Chagal|
|Samsung| NULL|     200| Allahabad|
|   Oppo|  170|     200|  Bhemghal|
|OnePlus|  140|     300|  Bhemghal|
| Realme| NULL|     300|     Delhi|
|   Iqoo|  170|     400|     Delhi|
+-------+-----+--------+----------+



In [ ]:
transDF.filter(transDF.price.isNotNull()).show()

+-------+--------+-----+-----+
|product|quantity|price|store|
+-------+--------+-----+-----+
|  Apple|     100|  250|    1|
|   Iqoo|     400|  170|    4|
|OnePlus|     300|  140|    2|
|     mi|     200|  150|    3|
|   Oppo|     200|  170|    2|
+-------+--------+-----+-----+



In [ ]:
transDF.fillna(140).show()

+-------+--------+-----+-----+
|product|quantity|price|store|
+-------+--------+-----+-----+
|Samsung|     200|  140|    1|
|   Iqoo|     400|  170|    4|
|  Apple|     100|  250|    1|
| Realme|     300|  140|    4|
|OnePlus|     300|  140|    2|
|     mi|     200|  150|    3|
|   Oppo|     200|  170|    2|
+-------+--------+-----+-----+



In [ ]:
location_df.explain(True)

== Parsed Logical Plan ==
'Sort ['quantity ASC NULLS FIRST], true
+- Project [product#0, price#2L, quantity#1L, store_name#26]
   +- Join Inner, (store#3L = store_id#25L)
      :- Repartition 5, true
      :  +- LogicalRDD [product#0, quantity#1L, price#2L, store#3L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [store_id#25L, store_name#26], false

== Analyzed Logical Plan ==
product: string, price: bigint, quantity: bigint, store_name: string
Sort [quantity#1L ASC NULLS FIRST], true
+- Project [product#0, price#2L, quantity#1L, store_name#26]
   +- Join Inner, (store#3L = store_id#25L)
      :- Repartition 5, true
      :  +- LogicalRDD [product#0, quantity#1L, price#2L, store#3L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [store_id#25L, store_name#26], false

== Optimized Logical Plan ==
Sort [quantity#1L ASC NULLS FIRST], true
+- Project [product#0, price#2L, quantity#1L, store_name#26]
   +- Join Inner, (store#3L = store_id#

In [ ]:
location_df.rdd.getNumPartitions()

1

In [ ]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

Snappy vs GZIP

In [ ]:
location_df.write.format('CSV').option('compression','snappy').save('/content/sample_data/test')


In [ ]:
location_df.write.format('CSV').option('compression','gzip').save('/content/sample_data/test/test2')

In [ ]:
!ls -lrth sample_data/test

total 8.0K
-rw-r--r-- 1 root root  137 Nov  5 11:35 part-00000-61052f39-9e1b-4c2e-863a-4c59d598d038-c000.csv.snappy
-rw-r--r-- 1 root root    0 Nov  5 11:35 _SUCCESS
drwxr-xr-x 2 root root 4.0K Nov  5 11:35 test2


In [ ]:
!ls -lrth sample_data/test/test2

total 4.0K
-rw-r--r-- 1 root root 123 Nov  5 11:35 part-00000-017807fc-057b-40a3-9a22-cdd60ae36ccc-c000.csv.gz
-rw-r--r-- 1 root root   0 Nov  5 11:35 _SUCCESS


    Pyspark SQL functions Split

In [ ]:
from pyspark.sql.functions import split

In [ ]:
transactions=[
    ('Apple-ipad-laptop',100,250,None),
    ('Samsung-mobile-watch',200,None,1),
    ('mi-earbud-sweep',200,150,3),
    ('OnePlus-earbud-switch',300,140,2),
    ('Oppo-camera-lens',None,170,2),
    ('Realme-bloot-specification',300,None,4),
    ('Iqoo-camera-loot',400,None,4),
    (None,200,None,None)
]
transDF=spark.createDataFrame(transactions,['product','quantity','price','store'])
transDF.show()

+--------------------+--------+-----+-----+
|             product|quantity|price|store|
+--------------------+--------+-----+-----+
|   Apple-ipad-laptop|     100|  250| NULL|
|Samsung-mobile-watch|     200| NULL|    1|
|     mi-earbud-sweep|     200|  150|    3|
|OnePlus-earbud-sw...|     300|  140|    2|
|    Oppo-camera-lens|    NULL|  170|    2|
|Realme-bloot-spec...|     300| NULL|    4|
|    Iqoo-camera-loot|     400| NULL|    4|
|                NULL|     200| NULL| NULL|
+--------------------+--------+-----+-----+



    Method1

In [ ]:
product=transDF.withColumn('category1',split('product','-').getItem(0)).withColumn('category2',split('product','-').getItem(1)).withColumn('category3',split('product','-').getItem(2))
product.show()

+--------------------+--------+-----+-----+---------+---------+-------------+
|             product|quantity|price|store|category1|category2|    category3|
+--------------------+--------+-----+-----+---------+---------+-------------+
|   Apple-ipad-laptop|     100|  250| NULL|    Apple|     ipad|       laptop|
|Samsung-mobile-watch|     200| NULL|    1|  Samsung|   mobile|        watch|
|     mi-earbud-sweep|     200|  150|    3|       mi|   earbud|        sweep|
|OnePlus-earbud-sw...|     300|  140|    2|  OnePlus|   earbud|       switch|
|    Oppo-camera-lens|    NULL|  170|    2|     Oppo|   camera|         lens|
|Realme-bloot-spec...|     300| NULL|    4|   Realme|    bloot|specification|
|    Iqoo-camera-loot|     400| NULL|    4|     Iqoo|   camera|         loot|
|                NULL|     200| NULL| NULL|     NULL|     NULL|         NULL|
+--------------------+--------+-----+-----+---------+---------+-------------+



    Method2

In [ ]:
split_column=split(transDF['product'],'-')
Product2=transDF.withColumn('category1',split_column.getItem(0)).withColumn('category2',split_column.getItem(1)).withColumn('category3',split_column.getItem(2))\
.drop('product')
Product2.show()

+--------+-----+-----+---------+---------+-------------+
|quantity|price|store|category1|category2|    category3|
+--------+-----+-----+---------+---------+-------------+
|     100|  250| NULL|    Apple|     ipad|       laptop|
|     200| NULL|    1|  Samsung|   mobile|        watch|
|     200|  150|    3|       mi|   earbud|        sweep|
|     300|  140|    2|  OnePlus|   earbud|       switch|
|    NULL|  170|    2|     Oppo|   camera|         lens|
|     300| NULL|    4|   Realme|    bloot|specification|
|     400| NULL|    4|     Iqoo|   camera|         loot|
|     200| NULL| NULL|     NULL|     NULL|         NULL|
+--------+-----+-----+---------+---------+-------------+



    Method3

In [ ]:
split_column=split(transDF['product'],'-')
e=transDF.select('product','quantity','price','store',split_column.getItem(0).alias('category1'),split_column.getItem(1).alias('category2'),split_column.getItem(2).alias('category3'))

In [ ]:
e.show()

+--------------------+--------+-----+-----+---------+---------+-------------+
|             product|quantity|price|store|category1|category2|    category3|
+--------------------+--------+-----+-----+---------+---------+-------------+
|   Apple-ipad-laptop|     100|  250| NULL|    Apple|     ipad|       laptop|
|Samsung-mobile-watch|     200| NULL|    1|  Samsung|   mobile|        watch|
|     mi-earbud-sweep|     200|  150|    3|       mi|   earbud|        sweep|
|OnePlus-earbud-sw...|     300|  140|    2|  OnePlus|   earbud|       switch|
|    Oppo-camera-lens|    NULL|  170|    2|     Oppo|   camera|         lens|
|Realme-bloot-spec...|     300| NULL|    4|   Realme|    bloot|specification|
|    Iqoo-camera-loot|     400| NULL|    4|     Iqoo|   camera|         loot|
|                NULL|     200| NULL| NULL|     NULL|     NULL|         NULL|
+--------------------+--------+-----+-----+---------+---------+-------------+



    Join(string)

In [ ]:
x=transDF.columns
stre=','.join(x)
print(stre)

product,quantity,price,store


In [ ]:
join_cond=" AND ".join(list(map(lambda y:('Target.'+y+'='+'source.'+y),x)))
print(join_cond)

Target.product=source.product AND Target.quantity=source.quantity AND Target.price=source.price AND Target.store=source.store


In [ ]:
location_df.groupby('store_name').agg({'quantity':'sum'}).show()

+----------+-------------+
|store_name|sum(quantity)|
+----------+-------------+
| Allahabad|          300|
|  Bhemghal|          500|
|     Delhi|          700|
|    Chagal|          200|
+----------+-------------+



In [ ]:
e.write.option('header',True).partitionBy('store').mode('overwrite').format('csv').save('/content/sample_data/test/test3')

In [ ]:
f=spark.read.option('header',True).csv('/content/sample_data/test/test3')
f.show()

+--------------------+--------+-----+---------+---------+-------------+-----+
|             product|quantity|price|category1|category2|    category3|store|
+--------------------+--------+-----+---------+---------+-------------+-----+
|Realme-bloot-spec...|     300| NULL|   Realme|    bloot|specification|    4|
|    Iqoo-camera-loot|     400| NULL|     Iqoo|   camera|         loot|    4|
|OnePlus-earbud-sw...|     300|  140|  OnePlus|   earbud|       switch|    2|
|Samsung-mobile-watch|     200| NULL|  Samsung|   mobile|        watch|    1|
|   Apple-ipad-laptop|     100|  250|    Apple|     ipad|       laptop| NULL|
|     mi-earbud-sweep|     200|  150|       mi|   earbud|        sweep|    3|
|    Oppo-camera-lens|    NULL|  170|     Oppo|   camera|         lens|    2|
|                NULL|     200| NULL|     NULL|     NULL|         NULL| NULL|
+--------------------+--------+-----+---------+---------+-------------+-----+



    Null values

In [ ]:
from pyspark.sql.functions import col,isnull,isnan,when,count
hNull=product.select([count(when(isnull(c),c)).alias(c) for c in product.columns])
hNull.show()

+-------+--------+-----+-----+---------+---------+---------+
|product|quantity|price|store|category1|category2|category3|
+-------+--------+-----+-----+---------+---------+---------+
|      1|       1|    4|    2|        1|        1|        1|
+-------+--------+-----+-----+---------+---------+---------+



    The End